In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = ':https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F56058%2F107505%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240416%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240416T063153Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6bc9cbf98bb73fc4538365a41cfd950871025c8c91908b1b09bdfb206fabb410755c17a2693482114a78b2ccf2f6cb2b007687bf1274a5ac83cf42a4594f41c558d365d0f0242d9dd5a8b0118d1508af97996cb474305c69ba65eee7073ce23cbc7f4389fb02da2dc82feac58fe2ecc1258a9325b7558e35e1f7e8ca2e9c2c81d04ce77df4fdc124c0ddf6e4fc77a0f0bcffc65437052441d01c2a04253067afef23d5f58bb02567257bf48372b8e72d54a45cce85b58840c3f7ae57ec1360666f8640b3a0270f848831962b4ad679140f931d4c4561cdb7c765c9901a2fb9c3e2cf2b8f5ef82488efb0d25329682b92f472dea29ae36cc448b444b933335262'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 5975957 bytes downloaded
Downloaded and uncompressed: 
Data source import complete.


# Cryptocurrency Predict using Recurrent Neural Network(RNN) and Convolutional Neural Network(CNN)
This code is from [pythonprogramming.net](https://pythonprogramming.net/cryptocurrency-recurrent-neural-network-deep-learning-python-tensorflow-keras/) tutorial.

Using Recurrent Neural Network (RNN) and Convolutional Neural Network (CNN), this code will predict the price movement of  Bitcoin, Ethereum, Litecoin and Bitcoin Cash cryptocurrency data. The target output will be a binary of **0** for price increase and **1** for price decrease. Also, this code will show the performace of RNN vs CNN.

In [ ]:
import numpy as np
import pandas as pd
import random
import time
from collections import deque
from sklearn import preprocessing

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization #CuDNNLSTM
from tensorflow.keras.layers import Activation, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

## Data

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/crypto_data/LTC-USD.csv", names = ['time','low','high','open','close','volume'])

df.head()

time        low       high       open      close      volume
0  1528968660  96.580002  96.589996  96.589996  96.580002    9.647200
1  1528968720  96.449997  96.669998  96.589996  96.660004  314.387024
2  1528968780  96.470001  96.570000  96.570000  96.570000   77.129799
3  1528968840  96.449997  96.570000  96.570000  96.500000    7.216067
4  1528968900  96.279999  96.540001  96.500000  96.389999  524.539978

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.shape

(101883, 6)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101883 entries, 0 to 101882
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   time    101883 non-null  int64  
 1   low     101883 non-null  float64
 2   high    101883 non-null  float64
 3   open    101883 non-null  float64
 4   close   101883 non-null  float64
 5   volume  101883 non-null  float64
dtypes: float64(5), int64(1)
memory usage: 4.7 MB


In [ ]:
df.columns

Index(['time', 'low', 'high', 'open', 'close', 'volume'], dtype='object')

## Parameters

A sequence lenght of 60 minutes data will be collected to be feed into the model and will predict the futere in 3 minute time.

In [ ]:
SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 3
RATIO_TO_PREDICT = 'LTC-USD'

## Classify Target

In [ ]:
def classify (current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

## Preprocessing Data

In [ ]:
def preprocess_df(df):
    df = df.drop('future', axis=1)

    for col in df.columns:
        if col != 'target':
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            df[col] = preprocessing.scale(df[col].values)
    df.dropna(inplace=True)

    sequential_data = []
    prev_days = deque(maxlen=SEQ_LEN)

    for i in df.values:
        prev_days.append([n for n in i[:-1]])
        if len(prev_days) == SEQ_LEN:
            sequential_data.append([np.array(prev_days), i[-1]])
    random.shuffle(sequential_data)

    buys = []
    sells = []

    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq, target])
        elif target == 1:
            buys.append([seq, target])
    random.shuffle(buys)
    random.shuffle(sells)

    lower = min(len(buys), len(sells))
    buys = buys[:lower]
    sells = sells[:lower]

    sequential_data = buys + sells
    random.shuffle(sequential_data)

    X = []
    y = []

    for seq, target in sequential_data:
        X.append(seq)
        y.append(target)

    return np.array(X),np.array(y)  #y  #added np array

## Build the Dataframe for data training and validation

In [ ]:
main_df = pd.DataFrame()
ratios = ['LTC-USD', 'BCH-USD', 'BTC-USD', 'ETH-USD']

for ratio in ratios:
    ratio = ratio.split('.csv')[0]
    dataset = f'/content/drive/MyDrive/crypto_data/{ratio}.csv'
    df = pd.read_csv(dataset, names=['time','low','high','open','close','volume'])
    df.rename(columns={'close':f'{ratio}_close','volume':f'{ratio}_volume'}, inplace=True)
    df.set_index('time', inplace=True)
    df = df[[f'{ratio}_close',f'{ratio}_volume']]

    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)

In [ ]:
main_df.fillna(method='ffill', inplace=True)
main_df.dropna(inplace=True)
print(main_df.head())

            LTC-USD_close  LTC-USD_volume  BCH-USD_close  BCH-USD_volume  \
time                                                                       
1528968720      96.660004      314.387024     870.859985       26.856577   
1528968780      96.570000       77.129799     870.099976        1.124300   
1528968840      96.500000        7.216067     870.789978        1.749862   
1528968900      96.389999      524.539978     870.000000        1.680500   
1528968960      96.519997       16.991997     869.989990        1.669014   

            BTC-USD_close  BTC-USD_volume  ETH-USD_close  ETH-USD_volume  
time                                                                      
1528968720    6487.379883        7.706374      486.01001       26.019083  
1528968780    6479.410156        3.088252      486.00000        8.449400  
1528968840    6479.410156        1.404100      485.75000       26.994646  
1528968900    6479.979980        0.753000      486.00000       77.355759  
1528968960    648

In [ ]:
main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)
main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'],main_df['future']))
main_df.dropna(inplace=True)
print(main_df.head())

            LTC-USD_close  LTC-USD_volume  BCH-USD_close  BCH-USD_volume  \
time                                                                       
1528968720      96.660004      314.387024     870.859985       26.856577   
1528968780      96.570000       77.129799     870.099976        1.124300   
1528968840      96.500000        7.216067     870.789978        1.749862   
1528968900      96.389999      524.539978     870.000000        1.680500   
1528968960      96.519997       16.991997     869.989990        1.669014   

            BTC-USD_close  BTC-USD_volume  ETH-USD_close  ETH-USD_volume  \
time                                                                       
1528968720    6487.379883        7.706374      486.01001       26.019083   
1528968780    6479.410156        3.088252      486.00000        8.449400   
1528968840    6479.410156        1.404100      485.75000       26.994646   
1528968900    6479.979980        0.753000      486.00000       77.355759   
1528968960 

In [ ]:
times = sorted(main_df.index.values)
last_5pct = sorted(main_df.index.values)[-int(0.05*len(times))]

print(time)
print(last_5pct)

<module 'time' (built-in)>
1534902300


In [ ]:
validation_main_df = main_df[(main_df.index >= last_5pct)]
train_main_df = main_df[(main_df.index < last_5pct)]

print(validation_main_df.head())
print(train_main_df.head())

            LTC-USD_close  LTC-USD_volume  BCH-USD_close  BCH-USD_volume  \
time                                                                       
1534902300      58.180000      380.746002     563.619995      100.205254   
1534902360      58.259998       56.717766     562.659973        7.565505   
1534902420      58.400002      702.115906     563.289978       20.834557   
1534902480      58.279999      247.673599     563.960022        4.556126   
1534902540      58.259998      243.763382     563.609985        1.140893   

            BTC-USD_close  BTC-USD_volume  ETH-USD_close  ETH-USD_volume  \
time                                                                       
1534902300    6779.990234       28.217054     293.250000      111.611511   
1534902360    6777.000000        8.318233     293.250000      125.616470   
1534902420    6802.830078       24.358656     294.200012      169.494324   
1534902480    6788.580078        6.864168     294.350006       88.935631   
1534902540 

In [ ]:
train_x, train_y = preprocess_df(train_main_df)
validation_x, validation_y = preprocess_df(validation_main_df.copy())

In [ ]:
print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"Dont buys: {np.sum(train_y == 0)}, buys: {np.sum(train_y == 1)}")
print(f"VALIDATION Dont buys: {np.sum(validation_y == 0)}, buys: {np.sum(validation_y == 1)}")

train data: 81638 validation: 4022
Dont buys: 40819, buys: 40819
VALIDATION Dont buys: 2011, buys: 2011


In [ ]:
# print(f"train data: {len(train_x)} validation: {len(validation_x)}")
# print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")
# print(f"VALIDATION Dont buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")

train data: 81638 validation: 4022


AttributeError: 'numpy.ndarray' object has no attribute 'count'

In [ ]:
print(train_x.shape[1:])

(60, 8)


## RNN Model

In [ ]:
EPOCHS = 7
BATCH_SIZE = 64
NAME = f'{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}'

In [ ]:
rnn_model = Sequential()

rnn_model.add(LSTM(128, input_shape=(train_x.shape[1:]),return_sequences=True))
rnn_model.add(Dropout(0.2))
rnn_model.add(BatchNormalization())

rnn_model.add(LSTM(128,return_sequences=True))
rnn_model.add(Dropout(0.1))
rnn_model.add(BatchNormalization())

rnn_model.add(LSTM(128))
rnn_model.add(Dropout(0.2))
rnn_model.add(BatchNormalization())

rnn_model.add(Dense(32, activation='relu'))
rnn_model.add(Dropout(0.2))

rnn_model.add(Dense(2, activation='softmax'))

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)

In [ ]:
rnn_model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
tensorboard = TensorBoard(log_dir='../{}'.format(NAME))

filepath = 'RNN_Final-{epoch:02d}-{val_accuracy:.3f}'

checkpoint = ModelCheckpoint('../{}.model'.format(filepath, monitor='val-acc', verbose=1, save_best_only=True, mode='max'))


start

In [ ]:
from keras.callbacks import TensorBoard

In [ ]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
history = rnn_model.fit(train_x, train_y,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(validation_x, validation_y),
                    callbacks = [tensorboard, checkpoint])

Epoch 1/7
1276/1276 [==============================] - ETA: 0s - loss: 0.6876 - accuracy: 0.5434

1276/1276 [==============================] - 718s 563ms/step - loss: 0.6876 - accuracy: 0.5434 - val_loss: 0.6821 - val_accuracy: 0.5515
Epoch 2/7
1276/1276 [==============================] - ETA: 0s - loss: 0.6870 - accuracy: 0.5477

1276/1276 [==============================] - 709s 556ms/step - loss: 0.6870 - accuracy: 0.5477 - val_loss: 0.6801 - val_accuracy: 0.5584
Epoch 3/7
1276/1276 [==============================] - ETA: 0s - loss: 0.6866 - accuracy: 0.5494

1276/1276 [==============================] - 711s 557ms/step - loss: 0.6866 - accuracy: 0.5494 - val_loss: 0.6834 - val_accuracy: 0.5567
Epoch 4/7
1276/1276 [==============================] - ETA: 0s - loss: 0.6855 - accuracy: 0.5514

1276/1276 [==============================] - 707s 554ms/step - loss: 0.6855 - accuracy: 0.5514 - val_loss: 0.6840 - val_accuracy: 0.5602
Epoch 5/7
1276/1276 [==============================] - ETA: 0s - loss: 0.6847 - accuracy: 0.5521

1276/1276 [==============================] - 707s 554ms/step - loss: 0.6847 - accuracy: 0.5521 - val_loss: 0.6819 - val_accuracy: 0.5654
Epoch 6/7
1276/1276 [==============================] - ETA: 0s - loss: 0.6837 - accuracy: 0.5546

1276/1276 [==============================] - 697s 546ms/step - loss: 0.6837 - accuracy: 0.5546 - val_loss: 0.6856 - val_accuracy: 0.5542
Epoch 7/7
1276/1276 [==============================] - ETA: 0s - loss: 0.6821 - accuracy: 0.5594

1276/1276 [==============================] - 721s 565ms/step - loss: 0.6821 - accuracy: 0.5594 - val_loss: 0.6855 - val_accuracy: 0.5733


In [ ]:
rnn_score = rnn_model.evaluate(validation_x, validation_y, verbose=0)
print('Test loss:', rnn_score[0])
print('Test accuracy:', rnn_score[1])

Test loss: 0.6855202317237854
Test accuracy: 0.5733466148376465


## CNN Model

In [ ]:
cnn_model = Sequential()

cnn_model.add(Conv1D(128,3,input_shape=(train_x.shape[1:])))
cnn_model.add(Activation('relu'))
cnn_model.add(Dropout(0.2))
cnn_model.add(MaxPooling1D(pool_size=2))

cnn_model.add(Conv1D(128,3))
cnn_model.add(Activation('relu'))
cnn_model.add(Dropout(0.2))
cnn_model.add(MaxPooling1D(pool_size=2))

cnn_model.add(Conv1D(128,3))
cnn_model.add(Activation('relu'))
cnn_model.add(Dropout(0.2))
cnn_model.add(MaxPooling1D(pool_size=2))

cnn_model.add(Flatten())
cnn_model.add(Dense(32))

cnn_model.add(Dense(2, activation='softmax'))

cnn_model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

cnn_history = cnn_model.fit(train_x, train_y,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(validation_x, validation_y))


Epoch 1/7
1276/1276 [==============================] - 66s 51ms/step - loss: 0.6990 - accuracy: 0.5112 - val_loss: 0.6927 - val_accuracy: 0.5077
Epoch 2/7
1276/1276 [==============================] - 66s 52ms/step - loss: 0.6930 - accuracy: 0.5178 - val_loss: 0.6924 - val_accuracy: 0.5112
Epoch 3/7
1276/1276 [==============================] - 64s 51ms/step - loss: 0.6924 - accuracy: 0.5194 - val_loss: 0.6922 - val_accuracy: 0.5085
Epoch 4/7
1276/1276 [==============================] - 67s 52ms/step - loss: 0.6922 - accuracy: 0.5205 - val_loss: 0.6924 - val_accuracy: 0.5020
Epoch 5/7
1276/1276 [==============================] - 64s 50ms/step - loss: 0.6920 - accuracy: 0.5231 - val_loss: 0.6929 - val_accuracy: 0.5114
Epoch 6/7
1276/1276 [==============================] - 66s 51ms/step - loss: 0.6915 - accuracy: 0.5222 - val_loss: 0.6920 - val_accuracy: 0.5139
Epoch 7/7
1276/1276 [==============================] - 63s 49ms/step - loss: 0.6913 - accuracy: 0.5242 - val_loss: 0.6930 - val_ac

In [ ]:
cnn_score = cnn_model.evaluate(validation_x, validation_y, verbose=0)
print('Test loss:', cnn_score[0])
print('Test accuracy:', cnn_score[1])

Test loss: 0.6930395364761353
Test accuracy: 0.5009945034980774
